In [ ]:
# https://towardsdatascience.com/land-cover-classification-using-tensorflow-in-python-791036eaa373
# https://github.com/phelber/EuroSAT

In [1]:
# CNN Training

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.keras.models import Sequential


In [2]:
# ===============================================================================
# plot styles

sns.set_theme(style="whitegrid")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica'

# ===============================================================================
# inspect data

train_data_dir = rf'{os.getcwd()}/data/2750'

data = {}
img_height, img_width = 0, 0
for path, dirs, files in os.walk(train_data_dir):
    dir_name = path.split('/')[-1]
    first_file = files[0]
    file_name, file_ext = first_file.split('.')
    if file_ext == 'jpg':
        image = Image.open(f'{path}/{first_file}')
        img_width, img_height = image.size
        data[dir_name] = {'count': len(files), 'height': img_height, 'width': img_width}

df = pd.DataFrame.from_dict(data, orient='index')
print(df.head(10))
# ax = df.plot.bar(y='count', rot=45)

# ===============================================================================
# create training, validation and testing dataset

batch_size = 32
train_dataset = image_dataset_from_directory(
    train_data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

validation_dataset = image_dataset_from_directory(
    train_data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# ===============================================================================
# inspect tensorflow training dataset

# get class names and number of classes
class_names = train_dataset.class_names
num_classes = len(class_names)
print(f'number of classes: {num_classes}', class_names)

# retrieve a single batch of 32 images.
image_batch, label_batch = next(iter(train_dataset))

# image_batch = tensor of the shape (32, 64, 64, 3), image shape = 64 x 64 x 3 (height x width x channels)
# label_batch = tensor of the shape (32,), (integer labels)
print(image_batch.shape)
print(label_batch.shape)

# check image value range
first_image = image_batch[0].numpy().astype("uint8")
print(np.min(first_image), np.max(first_image))

# visualize 10 images from dataset
plt.figure()
for i in range(10):
    ax = plt.subplot(2, 5, i + 1)
    image = image_batch[i].numpy().astype("uint8")
    plt.imshow(image)
    plt.title(class_names[label_batch[i]])
    plt.axis("off")
plt.show()

# ===============================================================================
# cache and prefetch for computational efficiency

# buffered prefetching yields data from disk without incurring I/O blocking
# Dataset.cache keeps the images in memory after they're loaded off disk during the first epoch
# Dataset.prefetch overlaps data preprocessing and model execution while training
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# ===============================================================================
# rescale image values into a range of [0,1]

# The RGB channel values are in the [0, 255] range
# Standardization rescales data to have a mean of 0 and a standard deviation of 1 (unit variance).

rescaling_layer = layers.experimental.preprocessing.Rescaling(
    scale=1. / 255,
    input_shape=(img_height, img_width, 3)
)

# rescale data outside of model
rescaled_ds = train_dataset.map(lambda x, y: (rescaling_layer(x), y))
image_batch, labels_batch = next(iter(rescaled_ds))
first_image = image_batch[0].numpy().astype("float32")
print(np.min(first_image), np.max(first_image))

# ===============================================================================
# define data augmentation layer

data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

# ===============================================================================
# build tensorflow sequential convolutional neural network model

model = Sequential([

    # layer 1 - preprocessing layer with convolution and max pooling
    rescaling_layer,
    layers.Conv2D(filters=8, kernel_size=3, padding='same',
                  activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"),

    # layer 2 - convolutional layer and max pooling
    layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"),

    # layer 3 - convolutional layer and max pooling
    layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"),

    # layer 4 - convolutional layer and max pooling
    layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"),
    layers.Dropout(rate=0.1),
    layers.Flatten(),

    # layer 5 - dense fully connected layer
    layers.Dense(units=128, activation='relu'),

    # layer 6 - output dense layer
    layers.Dense(num_classes)
])

# compile tensorflow sequential CNN model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# print model summary
model.summary()

# ===============================================================================
# train model

epochs = 1
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs
)
model.save(f'{os.getcwd()}/land_cover_model')

# ===============================================================================
# visualize training metrics

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'sns' is not defined